In [1]:
# Install: pip install yfinance
import yfinance as yf

def fetch_history(ticker_symbol, period="1mo", interval="1d"):
    """
    Fetches historical price data for the given ticker.

    Args:
      ticker_symbol (str): e.g. "AAPL"
      period (str): lookback period ("1d","5d","1mo","3mo","1y", etc.)
      interval (str): data granularity ("1m","5m","1h","1d", etc.)

    Returns:
      pandas.DataFrame: Open/High/Low/Close/Volume/etc.
    """
    ticker = yf.Ticker(ticker_symbol)
    df = ticker.history(period=period, interval=interval)
    return df

if __name__ == "__main__":
    df = fetch_history("AAPL", period="3mo", interval="1d")
    print(df.head())


YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [2]:
# Install if needed:
# !pip install yfinance

import pandas as pd
import yfinance as yf
from IPython.display import display

# 1) Define your tickers
symbols = ["AAPL", "MSFT", "GOOG", "AMZN"]

# 2) Batch‐download all at once
print("Downloading batch data from Yahoo Finance…")
batch_df = yf.download(
    tickers=symbols,
    period="3mo",
    interval="1d",
    group_by="ticker",    # keeps each ticker in its own sub‐DataFrame
    auto_adjust=False     # set True if you want adjusted prices
)

# 3) Display the first few rows for each symbol
for sym in symbols:
    print(f"\n— {sym} —")
    df = batch_df[sym].loc[:, ["Open", "High", "Low", "Close", "Volume"]]
    display(df.head())


[*********************100%***********************]  4 of 4 completed

4 Failed downloads:
['MSFT', 'AMZN', 'GOOG', 'AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')



— AAPL —


Price,Open,High,Low,Close,Volume
Date,,,,,



— MSFT —


Price,Open,High,Low,Close,Volume
Date,,,,,



— GOOG —


Price,Open,High,Low,Close,Volume
Date,,,,,



— AMZN —


Price,Open,High,Low,Close,Volume
Date,,,,,


In [ ]:
# Install yfinance if not already installed
import sys
!{sys.executable} -m pip install yfinance --quiet

import os
import pandas as pd
import yfinance as yf
import time
from math import ceil
from IPython.display import display

# Debug: show current working directory and CSV directory contents
print("Current working directory:", os.getcwd())
csv_dir = "csv"
print(f"Files in '{csv_dir}' directory:", os.listdir(csv_dir))

# Load valid Swedish companies
df_companies = pd.read_csv(os.path.join(csv_dir, "valid_swedish_company_data.csv"))
print("\nFirst few rows of company data:")
display(df_companies.head())

# Prepare ticker lists
swedish_symbols = df_companies['YahooTicker'].astype(str).str.upper().tolist()
yf_symbols = [f"{sym}.ST" for sym in swedish_symbols]

# Batch-check support
batch_size = 20
sleep_between_batches = 5  # seconds
supported = []
unsupported = []
total_batches = ceil(len(yf_symbols) / batch_size)

for i in range(total_batches):
    batch = yf_symbols[i*batch_size:(i+1)*batch_size]
    print(f"\nChecking batch {i+1}/{total_batches}: {batch}")
    try:
        batch_df = yf.download(
            tickers=batch,
            period="1d",
            interval="1d",
            group_by="ticker",
            threads=False,
            progress=False
        )
        for sym in batch:
            if sym in batch_df and not batch_df[sym].empty:
                supported.append(sym)
            else:
                unsupported.append(sym)
    except Exception as e:
        print(f"Batch download error: {e}")
        unsupported.extend(batch)
    time.sleep(sleep_between_batches)

# Compile results
results_df = pd.DataFrame({
    'Symbol': yf_symbols,
    'Supported': [sym in supported for sym in yf_symbols]
})
supported_df = results_df[results_df['Supported']]
unsupported_df = results_df[~results_df['Supported']]

print("\nSupported Swedish Tickers:")
display(supported_df)

print("\nUnsupported Swedish Tickers:")
display(unsupported_df)


In [ ]:
import os
import pandas as pd

# Print current working directory
print("Current working directory:", os.getcwd())

# List all files in the csv directory
csv_dir = "csv"
print(f"Files in {csv_dir} directory:", os.listdir(csv_dir))

# Now try to read the file using the relative path
df_companies = pd.read_csv(os.path.join(
    csv_dir, "valid_swedish_company_data.csv"))

# Display first few rows to confirm it loaded correctly
print(df_companies.head())